In [25]:
import pandas as pd
import datetime
import warnings
import psycopg2
import os

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

folder_path = r"S:/BI/data/Budgeting/"

# Получаем список всех файлов в папке с расширением .xlsx
files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Определяем последний изменённый файл
latest_file = max(files, key=os.path.getmtime)

# Получаем дату последнего изменения файла
timestamp = os.path.getmtime(latest_file)
created = datetime.datetime.fromtimestamp(timestamp).date()

data = pd.read_excel(latest_file).to_numpy()

conn = psycopg2.connect(dbname='', user="", password="", host='')

cur = conn.cursor()

created = datetime.date.today()

months = {15:"Январь", 17:"Февраль", 19:"Март", 21:"Апрель", 23:"Май", 25:"Июнь", 27:"Июль", 29:"Август", 31:"Сентябрь", 33:"Октябрь", 35:"Ноябрь", 37:"Декабрь"}

# Функция для создания даты на основе номера месяца
date_sql = {
        "Январь": '2024-01-01',
        "Февраль": '2024-02-01',
        "Март": '2024-03-01',
        "Апрель": '2024-04-01',
        "Май": '2024-05-01',
        "Июнь": '2024-06-01',
        "Июль": '2024-07-01',
        "Август": '2024-08-01',
        "Сентябрь": '2024-09-01',
        "Октябрь": '2024-10-01',
        "Ноябрь": '2024-11-01',
        "Декабрь": '2024-12-01'
}


for i in range(70, len(data)):
    row = data[i]
    l = []

    for j in range(len(row)):
        # Проверка на нули
        if pd.isnull(row[j]):
            l.append("Null")  # Используем None вместо "Null" для вставки в PostgreSQL
        else:
            l.append(row[j])

    title = l[2]
    
    work_type = l[8]

    if '70' in str(work_type):
        work_type = "ОСВОЕНИЕ ПЛАН"
    else:
        work_type = l[8]

    for q in months.keys():
        if l[q] is None: 
                value = "Null" 
        else: value = l[q]  # None для значений, где был "Null"
                
        date = date_sql[months[q]]
  # Дата месяца


        query = f"""INSERT INTO analitics_bi.plan_fact_monthly_bi VALUES ({title}, '{work_type}', '{months[q]}', {value}, '{created}', '{date}')"""

        cur.execute(query)

    conn.commit()


